In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-entities-for-ner/Entity Recognition in Resumes.json


In [24]:
import json
import pandas
import numpy as np
import pandas as pd
import spacy
from spacy.gold import biluo_tags_from_offsets
nlp = spacy.load("en_core_web_lg")

In [25]:

data_file = os.path.join("/kaggle/input/resume-entities-for-ner","Entity Recognition in Resumes.json")
 
with open(data_file,'r') as jfile:
    data = [json.loads(f_line) for f_line in jfile.readlines()]


In [26]:
df = pandas.DataFrame(data)
df['extras'].count()

0

In [27]:
#drop column extras as it has no value
df = df.drop('extras',axis=1)
df

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."
...,...,...
215,"Mansi Thanki\nStudent\n\nJamnagar, Gujarat - E...","[{'label': ['College Name'], 'points': [{'star..."
216,Anil Kumar\nMicrosoft Azure (Basic Management)...,"[{'label': ['Location'], 'points': [{'start': ..."
217,Siddharth Choudhary\nMicrosoft Office Suite - ...,"[{'label': ['Skills'], 'points': [{'start': 78..."
218,Valarmathi Dhandapani\nInvestment Banking Oper...,"[{'label': ['Skills'], 'points': [{'start': 92..."


In [28]:
len(df)

220

In [29]:
entities_map = {
    'Name': 'NAME', 
    'College Name': 'CLG',
    'Degree': 'DEG',
    'Graduation Year': 'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at': 'COMPANY',
    'Designation': 'DESIG',
    'Skills': 'SKILLS',
    'Location': 'LOC',
    'Email Address': 'EMAIL'
}

In [30]:
def solve_overlap(entity_loc):  #expected input [(start,end,entity),]
    '''
    sort the data with starting index
    condition1: if start of new is less then end of the old
                    if new and old has same entity
                        if old's end is less than new end
                            then update existing with (old_start, new_end, entity)
                    else
                        if old's end is less than new end
                            then update existing with (old_start, new_end, entity)
    '''
    entities = []
    sorted_entity = sorted(entity_loc,key = lambda a: a[0])
    for entity in sorted_entity:
        if not entities:
            entities.append(entity)
        else:
            prev = entities[-1]
            if entity[0] < prev[1]:
                if entity[2] == prev[2]:
                    if entity[1] > prev[1]:
                        entities[-1] = (prev[0],entity[1],entity[2])
                else:
                    if entity[1] > prev[1]:
                        entities[-1] = (prev[0], entity[1], entity[2])
            else:
                entities.append(entity)
    return entities

def create_entity(dataframe):
    entities = []    
    for i in range(len(df)):
        entity = []
        for annotation in df['annotation'][i]:
            try:
                label = entities_map.get(annotation['label'][0])
            except:
                pass
            #handling cases where entiry is not there in entity map
            if label:
                start = annotation['points'][0]['start']
                end = annotation['points'][0]['end']+1
                entity.append((start,end,label))
        processed_entity = solve_overlap(entity)
        entities.append(processed_entity)
    return entities

result = create_entity(df)
result[0]

[(0, 12, 'NAME'),
 (13, 46, 'DESIG'),
 (49, 58, 'COMPANY'),
 (60, 69, 'LOC'),
 (95, 146, 'EMAIL'),
 (372, 405, 'DESIG'),
 (407, 416, 'COMPANY'),
 (727, 770, 'DESIG'),
 (771, 814, 'CLG'),
 (856, 861, 'GRADYEAR'),
 (883, 905, 'CLG'),
 (939, 957, 'CLG'),
 (993, 1154, 'SKILLS'),
 (1295, 1622, 'SKILLS')]

In [31]:
df['entity'] = result
df

,content,annotation,entity
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, NAME), (13, 46, DESIG), (49, 58, COMP..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, NAME), (62, 68, LOC), (104, 148, EMAI..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, NAME), (22, 31, LOC), (65, 117, EMAIL..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, NAME), (13, 51, DESIG), (54, 60, COMP..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, NAME), (14, 22, DESIG), (24, 41, COMP..."
...,...,...,...
215,"Mansi Thanki\nStudent\n\nJamnagar, Gujarat - E...","[{'label': ['College Name'], 'points': [{'star...","[(0, 12, NAME), (13, 20, DESIG), (22, 30, LOC)..."
216,Anil Kumar\nMicrosoft Azure (Basic Management)...,"[{'label': ['Location'], 'points': [{'start': ...","[(0, 10, NAME), (11, 45, DESIG), (47, 52, LOC)..."
217,Siddharth Choudhary\nMicrosoft Office Suite - ...,"[{'label': ['Skills'], 'points': [{'start': 78...","[(0, 19, NAME), (20, 51, DESIG), (53, 62, LOC)..."
218,Valarmathi Dhandapani\nInvestment Banking Oper...,"[{'label': ['Skills'], 'points': [{'start': 92...","[(0, 21, NAME), (22, 56, DESIG), (57, 66, LOC)..."


In [32]:
def create_train_Data(dataframe):
    sentences = []
    tags = []
    for i in range(len(df)):
        text = df['content'][i]
        #text = text.replace('\n',' ')
        entities = df['entity'][i]
        doc = nlp(text)
        #print([token.text for token in doc])
        tag = biluo_tags_from_offsets(doc,entities)
        #combining token and corresponding tags
        tag_df = pd.DataFrame([list(doc),tag]).T
        #get the end of a sentence within a doc by validating . and O
        period_loc = []
        for i in range(len(tag)):
            if tag_df[0][i].text is '.' and tag_df[1][i] is 'O':
                period_loc.append(i)
        #adding the last loc
        period_loc.append(len(doc))

        #extract sentences based on period.

        last = 0
        datas = []
        for i in period_loc:
            datas.append([list(doc)[last:i], tag[last:i]])
            last = i

        #validate and replace tag '-' with 'O'

        for data in datas:
            modified_tag = ['O' if t is '-' else t for t in data[1]]
            if len(set(modified_tag))>1:
                sentences.append(data[0])
                tags.append(modified_tag)

    return (sentences, tags)
    

In [33]:
import warnings
warnings.filterwarnings('ignore')

In [34]:
res = create_train_Data(df)
print(len(res[0]),len(res[1]))

764 764


In [35]:
res[1][0]

['B-NAME',
 'L-NAME',
 'O',
 'B-DESIG',
 'I-DESIG',
 'L-DESIG',
 'O',
 'U-COMPANY',
 'O',
 'U-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [36]:
all_tags = set(['X','[CLS]','[SEP]'])
for i in range(len(res[1])):
    all_tags = all_tags.union(res[1][i])
all_tags

{'B-CLG',
 'B-COMPANY',
 'B-DEG',
 'B-DESIG',
 'B-EMAIL',
 'B-GRADYEAR',
 'B-LOC',
 'B-NAME',
 'B-SKILLS',
 'B-YOE',
 'I-CLG',
 'I-COMPANY',
 'I-DEG',
 'I-DESIG',
 'I-EMAIL',
 'I-LOC',
 'I-NAME',
 'I-SKILLS',
 'I-YOE',
 'L-CLG',
 'L-COMPANY',
 'L-DEG',
 'L-DESIG',
 'L-EMAIL',
 'L-GRADYEAR',
 'L-LOC',
 'L-NAME',
 'L-SKILLS',
 'L-YOE',
 'O',
 'U-CLG',
 'U-COMPANY',
 'U-DEG',
 'U-DESIG',
 'U-EMAIL',
 'U-GRADYEAR',
 'U-LOC',
 'U-SKILLS',
 'U-YOE',
 'X',
 '[CLS]',
 '[SEP]'}

In [37]:
tag2indx = {t:i for i, t in enumerate(all_tags)}
tag2indx

{'B-YOE': 0,
 'L-DESIG': 1,
 'U-LOC': 2,
 'B-DESIG': 3,
 'I-EMAIL': 4,
 'B-LOC': 5,
 'U-DESIG': 6,
 'B-COMPANY': 7,
 'U-SKILLS': 8,
 'L-COMPANY': 9,
 'I-LOC': 10,
 'X': 11,
 'L-GRADYEAR': 12,
 'U-YOE': 13,
 'L-SKILLS': 14,
 '[CLS]': 15,
 'B-SKILLS': 16,
 'B-DEG': 17,
 'L-CLG': 18,
 'B-GRADYEAR': 19,
 'L-NAME': 20,
 '[SEP]': 21,
 'I-DESIG': 22,
 'U-EMAIL': 23,
 'L-YOE': 24,
 'L-EMAIL': 25,
 'L-DEG': 26,
 'B-EMAIL': 27,
 'U-DEG': 28,
 'I-NAME': 29,
 'I-SKILLS': 30,
 'U-CLG': 31,
 'I-COMPANY': 32,
 'L-LOC': 33,
 'I-CLG': 34,
 'B-CLG': 35,
 'I-DEG': 36,
 'U-COMPANY': 37,
 'U-GRADYEAR': 38,
 'I-YOE': 39,
 'O': 40,
 'B-NAME': 41}

In [38]:
indx2tag = {i:t for i,t in enumerate(all_tags)}
indx2tag

{0: 'B-YOE',
 1: 'L-DESIG',
 2: 'U-LOC',
 3: 'B-DESIG',
 4: 'I-EMAIL',
 5: 'B-LOC',
 6: 'U-DESIG',
 7: 'B-COMPANY',
 8: 'U-SKILLS',
 9: 'L-COMPANY',
 10: 'I-LOC',
 11: 'X',
 12: 'L-GRADYEAR',
 13: 'U-YOE',
 14: 'L-SKILLS',
 15: '[CLS]',
 16: 'B-SKILLS',
 17: 'B-DEG',
 18: 'L-CLG',
 19: 'B-GRADYEAR',
 20: 'L-NAME',
 21: '[SEP]',
 22: 'I-DESIG',
 23: 'U-EMAIL',
 24: 'L-YOE',
 25: 'L-EMAIL',
 26: 'L-DEG',
 27: 'B-EMAIL',
 28: 'U-DEG',
 29: 'I-NAME',
 30: 'I-SKILLS',
 31: 'U-CLG',
 32: 'I-COMPANY',
 33: 'L-LOC',
 34: 'I-CLG',
 35: 'B-CLG',
 36: 'I-DEG',
 37: 'U-COMPANY',
 38: 'U-GRADYEAR',
 39: 'I-YOE',
 40: 'O',
 41: 'B-NAME'}

In [108]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [40]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [41]:
!pip install transformers

In [169]:
import transformers
from transformers import DistilBertTokenizer, BertForTokenClassification

In [170]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [172]:
device = torch.device('cuda')

In [173]:
sentences, tags = res[0], res[1]

In [174]:
def format_trainable_data(sentences, tags):
    train_data = []
    train_tag = []
    for sent, tag in zip(sentences, tags):
        train_text =['[CLS]']
        train_label = ['[CLS]']
        for word, t in zip(sent,tag):
            tokens = tokenizer.tokenize(word.text)
            for i,token in enumerate(tokens):
                train_text.append(token)
                if i==0:
                    train_label.append(t)
                else:
                    train_label.append('X')
        train_text.append('[SEP]')
        train_label.append('[SEP]')
        train_data.append(train_text)
        train_tag.append(train_label)
    return train_data, train_tag

In [175]:
train_X, train_Y = format_trainable_data(sentences, tags)

In [176]:
train_Y[0]

['[CLS]',
 'B-NAME',
 'X',
 'X',
 'X',
 'X',
 'L-NAME',
 'X',
 'B-DESIG',
 'I-DESIG',
 'L-DESIG',
 'O',
 'U-COMPANY',
 'X',
 'X',
 'X',
 'U-LOC',
 'X',
 'O',
 'O',
 'O',
 'O',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [177]:
MAX_LEN = 512
BATCH = 6

In [178]:
tokenized_train_X = pad_sequences([tokenizer.convert_tokens_to_ids(tokens) for tokens in train_X],maxlen= MAX_LEN, dtype= "long", padding= "post", truncating= "post")

In [179]:
tokenized_train_Y = pad_sequences([[tag2indx[t] for t in tag] for tag in train_Y], maxlen=MAX_LEN, dtype="long",padding='post',truncating='post',value=tag2indx['O'])

In [180]:
tokenized_train_Y[0]

array([15, 41, 11, 11, 11, 11, 20, 11,  3, 22,  1, 40, 37, 11, 11, 11,  2,
       11, 40, 40, 40, 40, 11, 40, 40, 40, 40, 40, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 11, 40, 40, 40, 40, 40, 21,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40

In [181]:
attention_mask = [[float(val>0) for val in m] for m in tokenized_train_X]
print(attention_mask[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [182]:
train_x, val_x, train_y, val_y, train_attention_mask, val_attention_mask = train_test_split(tokenized_train_X, tokenized_train_Y, attention_mask, test_size=0.3, random_state=2020)

In [183]:
train_y[2]

array([15, 40, 40, 11, 40, 40, 40, 40, 40, 40, 40, 40, 11, 11, 40, 11, 11,
       11, 11, 11, 11,  3, 22,  1, 37, 11, 11, 11, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 11, 40, 40, 11, 21, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40

In [184]:
train_sent = torch.tensor(train_x)
val_sent = torch.tensor(val_x)
train_label = torch.tensor(train_y)
val_label = torch.tensor(val_y)
train_attention = torch.tensor(train_attention_mask)
val_attention = torch.tensor(val_attention_mask)

In [185]:
training_data = TensorDataset(train_sent, train_label, train_attention)
train_sampler = RandomSampler(training_data)
train_dataloader = DataLoader(training_data, sampler = train_sampler, batch_size = BATCH)

In [186]:
validation_data = TensorDataset(val_sent, val_label, val_attention)
val_sampler = SequentialSampler(validation_data)
val_dataloader = DataLoader(validation_data, sampler = val_sampler, batch_size = BATCH )

In [151]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels = len(tag2indx))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [191]:
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-cased", num_labels = len(tag2indx))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [192]:
model.cuda()

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [193]:
full_weight_training = True
if full_weight_training:
    bert_params = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params':[p for n, p  in bert_params if not any(nd in n for nd in no_decay)],
        'weight_decay_rate':0.01},
        {'params':[p for n, p in bert_params if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{'params':[p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters,lr= 3e-5)

NameError: name 'train_dataloader' is not defined

In [196]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
epochs = 10
max_grad_norm = 1.0
for _ in range(epochs):
    model.train()
    tr_loss = 0
    num_tr_ex, num_tr_steps = 0, 0
    for step, batches in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batches)
        input_data, input_label, input_mask = batch
        output = model(input_data, attention_mask = input_mask, labels = input_label)
        loss = output.loss
        #print(loss)
        loss.backward()
        tr_loss += loss.item()
        num_tr_ex += input_data.size(0)
        num_tr_steps += 1
        torch.nn.utils.clip_grad_norm(model.parameters(),max_norm=max_grad_norm)
        optimizer.step()
        model.zero_grad()
    print(f"training loss: {tr_loss/num_tr_steps}")

training loss: 0.05839630239381549
training loss: 0.04376428159414215
training loss: 0.03300733588049921
training loss: 0.029638902702776904
training loss: 0.02404730509589897
training loss: 0.02224283708845464
training loss: 0.01870957000463615
training loss: 0.01503400238665093
training loss: 0.014329073354183288
training loss: 0.011865901966118829


In [198]:
!pip install --no-deps seqeval[gpu]

     |████████████████████████████████| 43 kB 268 kB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=07c620b8f5a3a6a150d4dafb89eeb9f2c4ec8e52865531f468fe565cf71d8f04
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [199]:
from seqeval.metrics import classification_report, accuracy_score, f1_score

In [206]:
model.eval()

y_true = []
y_pred = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, label_ids, input_mask  = batch

    with torch.no_grad():
        out = model(input_ids, attention_mask=input_mask)
        logits = out.logits

    logits = logits.detach().cpu().numpy()
    logits = [list(p) for p in np.argmax(logits, axis=2)]
    
    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
        temp_1 = [] # Real one
        temp_2 = [] # Predict one
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if indx2tag[label_ids[i][j]] != "X" and indx2tag[label_ids[i][j]] != "[CLS]" and indx2tag[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(indx2tag[label_ids[i][j]])
                    temp_2.append(indx2tag[logits[i][j]])
            else:
                 break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)
    
print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

print(classification_report(y_true, y_pred,digits=4))

f1 socre: 0.396677
Accuracy score: 0.805003
              precision    recall  f1-score   support

         CLG     0.1111    0.2459    0.1531        61
     COMPANY     0.4778    0.5695    0.5196       151
         DEG     0.2899    0.3704    0.3252        54
       DESIG     0.3865    0.5943    0.4684       106
       EMAIL     0.2353    0.4800    0.3158        25
    GRADYEAR     0.4366    0.5536    0.4882        56
         LOC     0.5570    0.7094    0.6241       117
        NAME     0.8833    0.8983    0.8908        59
      SKILLS     0.0667    0.1288    0.0879       132
         YOE     0.5000    0.1818    0.2667        11
           _     0.0000    0.0000    0.0000         0

   micro avg     0.3310    0.4948    0.3967       772
   macro avg     0.3586    0.4302    0.3763       772
weighted avg     0.3853    0.4948    0.4279       772

